<a href="https://colab.research.google.com/github/HauwaUmar/DATA-6100/blob/main/Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTING RELEVANT LIBRARIES

In [33]:
import numpy as np
import pandas as pd
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBRegressor 
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing

IMPORTING DATASET 

In [4]:
train_dataset = pd.read_csv("/train.csv")
test_dataset = pd.read_csv("/test.csv")
test_target = pd.read_csv("/sample_submission.csv")

In [6]:
train_dataset.shape

(1460, 81)

In [7]:
test_dataset.shape

(1459, 80)

In [8]:
test_target.shape

(1459, 2)

DATA EXPLORATION / ANALYSIS

In [9]:
# number of missing values in each column
missing_val_train = train_dataset.isnull().sum()
missing_val_test = test_dataset.isnull().sum()

In [11]:
pd.set_option('display.max_rows', missing_val_train.shape[0]+1)
missing_val_train

Id                  0
MSSubClass          0
MSZoning            0
LotFrontage       259
LotArea             0
Street              0
Alley            1369
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType          8
MasVnrArea          8
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           37
BsmtCond           37
BsmtExposure       38
BsmtFinType1       37
BsmtFinSF1          0
BsmtFinType2       38
BsmtFinSF2          0
BsmtUnfSF           0
TotalBsmtSF         0
Heating             0
HeatingQC           0
CentralAir          0
Electrical          1
1stFlrSF            0
2ndFlrSF            0
LowQualFin

In [12]:
pd.set_option('display.max_rows', missing_val_test.shape[0]+1)
missing_val_test

Id                  0
MSSubClass          0
MSZoning            4
LotFrontage       227
LotArea             0
Street              0
Alley            1352
LotShape            0
LandContour         0
Utilities           2
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         1
Exterior2nd         1
MasVnrType         16
MasVnrArea         15
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           44
BsmtCond           45
BsmtExposure       44
BsmtFinType1       42
BsmtFinSF1          1
BsmtFinType2       42
BsmtFinSF2          1
BsmtUnfSF           1
TotalBsmtSF         1
Heating             0
HeatingQC           0
CentralAir          0
Electrical          0
1stFlrSF            0
2ndFlrSF            0
LowQualFin

In [13]:
# Going through both data sets (i.e training and testing) it is clear that some columns have a lot of missing vlaues 
# removing columns with a total of missing values higher than 1000 
no_null_train_dataset = train_dataset.dropna(axis=1,thresh=1000)

no_null_test_dataset = test_dataset.dropna(axis=1,thresh=1000)


In [14]:
no_null_train_dataset.shape

(1460, 76)

In [15]:
no_null_test_dataset.shape

(1459, 75)

In [16]:
# shows the percentage of missing values 
def info_on_missing_values(df):
    total_missing_values = df.isnull().sum().sort_values(ascending = False)[df.isnull().sum().sort_values(ascending = False) != 0]
    percentage = round(df.isnull().sum().sort_values(ascending = False)/len(df)*100,2)[round(df.isnull().sum().sort_values(ascending = False)/len(df)*100,2) != 0]
    data_type = df.dtypes.sort_values(ascending = False)[df.isnull().sum().sort_values(ascending = False) != 0]
    decription_of_missing_values = pd.concat([total_missing_values, percentage, data_type], axis=1, keys=['Total Missing Val','Percentage','Data Type'])
    return decription_of_missing_values

In [17]:
info_on_missing_values(no_null_train_dataset)

,Total Missing Val,Percentage,Data Type
LotFrontage,259,17.74,float64
GarageType,81,5.55,object
GarageCond,81,5.55,object
GarageYrBlt,81,5.55,float64
GarageFinish,81,5.55,object
GarageQual,81,5.55,object
BsmtFinType2,38,2.60,object
BsmtExposure,38,2.60,object
BsmtFinType1,37,2.53,object
BsmtCond,37,2.53,object


In [18]:
info_on_missing_values(no_null_test_dataset)

,Total Missing Val,Percentage,Data Type
LotFrontage,227,15.56,float64
GarageYrBlt,78,5.35,float64
GarageFinish,78,5.35,object
GarageQual,78,5.35,object
GarageCond,78,5.35,object
GarageType,76,5.21,object
BsmtCond,45,3.08,object
BsmtExposure,44,3.02,object
BsmtQual,44,3.02,object
BsmtFinType1,42,2.88,object


In [19]:
# OBSERVATIONS/ASSUMPTIONS
# Going through the dataset description some columns have NA which reoresent none not missing value
# manually listing columns where na is actually none 
none_columns = ['BsmtQual', 'BsmtCond', 'BsmtExposure','BsmtFinType1','BsmtFinType2', 'GarageType', 'GarageFinish','GarageQual','GarageCond']


In [21]:
# copying datafram from no_null_test_dataset and no_null_train_dataset 
test_dataset = no_null_test_dataset.copy()
train_dataset = no_null_train_dataset.copy()

In [43]:
#drop id column for both test and train dataset
train_dataset = train_dataset.drop(['Id'],axis = 1)
test_dataset = test_dataset.drop(['Id'],axis = 1)

In [45]:
train_dataset.shape, test_dataset.shape

((1460, 75), (1459, 74))

DATA CLEANING 

In [49]:
# replacing NA with none in observed columns
for i in none_columns:
    train_dataset[i] = train_dataset[i].fillna('None')
    test_dataset[i] = test_dataset[i].fillna('None')


In [50]:
#show columns with missing values in train dataset
info_on_missing_values(train_dataset)

,Total Missing Val,Percentage,Data Type


In [51]:
#show columns with missing values in test dataset
info_on_missing_values(test_dataset)


,Total Missing Val,Percentage,Data Type


In [52]:
# list of columns with missing values in train dataset
missing_train_cols = train_dataset.columns[train_dataset.isna().any()]
missing_train_cols

Index([], dtype='object')

In [53]:
# list of columns with missing values in train dataset
missing_test_cols = test_dataset.columns[test_dataset.isna().any()]
missing_test_cols

Index([], dtype='object')

In [28]:
# FILLS THE REMAINING COLUMNS THAT HAVE NA WITH THE MODE OF THE COLUMN IN TRAIN DATASET
for col in missing_train_cols:
    train_dataset[col] = train_dataset[col].fillna(train_dataset[col].mode()[0])



In [29]:
# FILLS THE REMAINING COLUMNS THAT HAVE NA WITH THE MODE OF THE COLUMN IN TRAIN DATASET
for a in missing_test_cols:
    test_dataset[a] = test_dataset[a].fillna(test_dataset[a].mode()[0])


In [30]:
#check for missing value in train dataset
info_on_missing_values(train_dataset)

,Total Missing Val,Percentage,Data Type


In [32]:
#check for missing value in test dataset
info_on_missing_values(test_dataset)

,Total Missing Val,Percentage,Data Type


DATA PREPROCESSING

In [46]:
train_dataset.shape

(1460, 75)

In [47]:
test_dataset.shape

(1459, 74)

In [48]:
# ASSIGNING X AND Y VALUES FOR TRAIN DATASET
Y_train = train_dataset['SalePrice']
X_train = train_dataset.drop(['SalePrice'],axis = 1)


In [55]:
# ASSIGNING X AND Y VALUES FOR TEST DATASET
X_test = test_dataset.copy()
Y_test = test_target.drop(['Id'],axis = 1)

In [56]:
Y_test

,SalePrice
0,169277.052498
1,187758.393989
2,183583.683570
3,179317.477511
4,150730.079977
...,...
1454,167081.220949
1455,164788.778231
1456,219222.423400
1457,184924.279659


In [58]:
X_train.shape,X_test.shape

((1460, 74), (1459, 74))

In [60]:
# the label encoder object
le = preprocessing.LabelEncoder()


In [62]:
# LABELLING CATGEORICAL DATA  USING LABEL ENCODER
train_object_col = list(X_train.select_dtypes(include='object'))
train_object_col

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']